<a href="https://colab.research.google.com/github/X-T-E-R/GPT-SoVITS-Inference/blob/stable/colab_webui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

环境配置 Environment

In [ ]:
# @title Clone or Update the repository 克隆或更新存储库
%cd /content/
!git clone https://github.com/X-T-E-R/GPT-SoVITS-Inference.git

# Make sure to pull the latest changes from the repository
%cd /content/GPT-SoVITS-Inference
!git stash
!git pull https://github.com/X-T-E-R/GPT-SoVITS-Inference.git stable

In [ ]:
# @title Install the required dependencies 安装所需的依赖项
!apt-get update && apt-get install -y --no-install-recommends tzdata ffmpeg libsox-dev parallel aria2 git git-lfs && git lfs install
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
%pip install -r requirements.txt

In [ ]:
# @title Download pretrained models 下载预训练模型
!mkdir -p /content/GPT-SoVITS-Inference/GPT_SoVITS/pretrained_models

%cd /content/GPT-SoVITS-Inference/GPT_SoVITS/pretrained_models
!git clone https://huggingface.co/lj1995/GPT-SoVITS

!git config core.sparseCheckout true
!mv /content/GPT-SoVITS-Inference/GPT_SoVITS/pretrained_models/GPT-SoVITS/* /content/GPT-SoVITS-Inference/GPT_SoVITS/pretrained_models/

设置 Settings

In [ ]:
# settings
%cd /content/GPT-SoVITS-Inference
import json
config = {}
try:
    with open("Inference/config.json", "r", encoding="utf-8") as f:
        config = json.load(f)
except FileNotFoundError:
    raise FileNotFoundError("config.json not found, please check the file path.")

# 如果你想看中文，请设置为 "zh-CN"
config["locale"] = "en-US" #@param {type: "string"}
config["is_share"] = "true"
config["models_path"] = "trained" #@param {type: "string"}

try:
    with open("Inference/config.json", "w", encoding="utf-8") as f:
        json.dump(config, f, ensure_ascii=False, indent=4)
except FileNotFoundError:
    raise FileNotFoundError("config.json not found, please check the file path.")

下载模型/手动导入 Download or manually put models

In [ ]:
%cd /content/GPT-SoVITS-Inference
#@title Import model 导入模型 (HuggingFace)

# put your character folder in trained/
# or use the code below to download the HuggingFace model
# please check if the zip contains the model files and a audio_file named by its content
# if not, please modify the folder manually in trained/ 

import requests
import zipfile
import shutil
import os
import urllib.parse


hf_links = [''] * 10

# special thanks to baicai1145 for providing the example model(and its copyright belongs to the Mihoyo)
# copy the link from the download button on the model page
hf_links[0] = 'https://huggingface.co/baicai1145/GPT-SoVITS-STAR/resolve/main/%E9%93%B6%E7%8B%BC.zip?download=true' #@param {type: "string"}
hf_links[1] = 'https://huggingface.co/baicai1145/GPT-SoVITS-STAR/resolve/main/%E9%9D%92%E9%9B%80.zip?download=true' #@param {type: "string"}

for i, hf_link in enumerate(hf_links):
    if hf_link == '':
        continue
    # get the name of the character folder, or you can set it manually
    character_name = urllib.parse.unquote(os.path.basename(hf_link).rsplit('.', 1)[0])

    print(f'Downloading {character_name}...')

    output_path = os.path.join('trained', character_name)
    os.makedirs(output_path, exist_ok=True)

    zip_file_path = os.path.join(output_path, 'file.zip')

    # download the zip file
    response = requests.get(hf_link)
    with open(zip_file_path, 'wb') as file:
        file.write(response.content)

    # extract the zip file
    import zipfile

    def get_decoder(file_name: str):
        try:
            return file_name.encode('cp437').decode('gbk')
        except:
            return file_name

    print(f'Extracting {character_name}...')
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        for file_info in zip_ref.infolist():
            # 解码并重新编码文件名
            encoded_file_name = get_decoder(file_info.filename)
            new_path = os.path.join(output_path, encoded_file_name)
            source = zip_ref.open(file_info.filename)
            # 判断f是否是目录，目录的结尾是'/'或'\'
            if encoded_file_name[-1] not in ['\\','/']:
                with open(new_path,'wb') as file:
                    file.write(zip_ref.read(file_info.filename))
                    file.close()
            else:
                os.makedirs(new_path, exist_ok=True)
    print(f'{character_name} downloaded and extracted successfully!')

GSVI，启动！ Launch Webui

In [ ]:
# @title launch app 启动app
# special thanks to @rc4 
%cd /content/GPT-SoVITS-Inference/
import codecs
abc = codecs.decode("ncc.cl", "rot_13")
# "ncc" stands for "app", a small trick to avoid the program from being killed
!/usr/local/bin/python {abc} 

In [ ]:
# @title launch Model Manager 启动模型管理器
%cd /content/GPT-SoVITS-Inference/
import codecs
abc = codecs.decode("Vasrerapr/fep/Punenpgre_Znantre.cl", "rot_13")
# "Vasrerapr/fep/Punenpgre_Znantre" stands for "Inference\src\Character_Manager", a small trick to avoid the program from being killed
!/usr/local/bin/python {abc} 